In [19]:
import pandas as pd
import pickle
from catboost import CatBoostRegressor, Pool
from sklearn.ensemble import AdaBoostRegressor
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, mean_absolute_error

In [20]:
# Use only last 3 datasets for fitting models

#df1 = pd.read_csv('datasets/db_waw_flats.csv', parse_dates = ['today', 'publication_date'])
#df2 = pd.read_csv('datasets/db_waw_flats_17_06.csv', parse_dates = ['today', 'publication_date'])
#df3 = pd.read_csv('datasets/db_waw_flats_27_6.csv', parse_dates = ['today', 'publication_date'])
#df4 = pd.read_csv('datasets/db_waw_flats_10_7.csv', parse_dates = ['today', 'publication_date'])
#df5 = pd.read_csv('datasets/db_waw_flats_27_7.csv', parse_dates = ['today', 'publication_date'])
#df6 = pd.read_csv('datasets/db_waw_flats_3_8.csv', parse_dates = ['today', 'publication_date'])
#df7 = pd.read_csv('datasets/db_waw_flats_16_8.csv', parse_dates = ['today', 'publication_date'])
df7 = pd.read_csv('datasets/db_waw_flats_4_10.csv', parse_dates = ['today', 'publication_date'])
df8 = pd.read_csv('datasets/db_waw_flats_10_10.csv', parse_dates = ['today', 'publication_date'])
df9 = pd.read_csv('datasets/db_waw_flats_15_10.csv', parse_dates = ['today', 'publication_date'])

In [21]:
common_df = pd.concat([df7,df8,df9], ignore_index=True)
common_df

,price,area,rooms,renovation,floor,market,elevator,street,district,balcony,terrace,garden,parking,central_heating,seller,blok,cena_m,today,publication_date
0,1177777.0,50.00,3.0,3,2,2,1,ul. stefana żeromskiego,bielany,1,0,0,1,1,2,0,23555.540000,2023-10-04,2023-10-04 10:49:42
1,789000.0,56.90,3.0,3,2,2,1,ul. zygmunta modzelewskiego,mokotów,1,0,0,0,0,1,0,13866.432337,2023-10-04,2023-09-26 11:40:49
2,899999.0,53.00,2.0,1,2,2,1,ul. grażyny,mokotów,1,0,0,0,0,1,0,16981.113208,2023-10-04,2023-09-26 14:40:53
3,777000.0,53.31,3.0,3,2,2,1,zygmunta modzelewskiego,mokotów,0,0,0,0,1,1,1,14575.126618,2023-10-04,2023-07-04 12:22:57
4,2599000.0,104.42,4.0,3,2,2,1,ul. konstruktorska,mokotów,1,0,0,1,1,2,0,24889.867841,2023-10-04,2023-08-11 16:54:32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22674,5049000.0,159.00,4.0,3,2,2,1,pl. plac politechniki,śródmieście,1,0,0,0,1,1,0,31754.716981,2023-10-15,2018-12-20 16:00:09
22675,1479400.0,113.80,4.0,3,1,2,1,ul. meander,ursynów,1,0,0,1,1,1,1,13000.000000,2023-10-15,2015-10-27 20:02:18
22676,NaN,319.00,2.0,3,2,2,0,stara miłosna,wesoła,0,0,0,1,0,1,0,NaN,2023-10-15,2015-02-06 23:10:16
22677,NaN,262.00,6.0,3,1,2,0,stary żoliborz,żoliborz,0,0,0,1,0,1,0,NaN,2023-10-15,2015-02-06 22:28:39


In [22]:
common_df.drop(columns=['today']).drop_duplicates(keep='last', inplace=True)

In [23]:
cols_x = ['district', 'area', 'rooms', 'renovation', 'floor', 'balcony', 'terrace',
          'garden', 'parking', 'central_heating', 'market', 'seller', 'blok', 'elevator']

cat_cols = ['district', 'renovation', 'floor', 'balcony', 'terrace', 'garden', 'parking',
            'central_heating', 'market', 'seller', 'blok', 'elevator']
col_y = ['price']

model_df = common_df[cols_x + col_y].dropna()

In [24]:
#le = preprocessing.LabelEncoder()
#le.fit(model_df['district'])
le = pickle.load(open('district_encode', 'rb'))
model_df['district'] = le.transform(model_df['district'])

In [25]:
X_train, X_test, y_train, y_test = train_test_split(model_df[cols_x], model_df['price'], test_size=0.2, shuffle=False)

In [26]:
from sklearn.tree import DecisionTreeRegressor
est = DecisionTreeRegressor(random_state=0, max_depth=12)
ada_model = AdaBoostRegressor(estimator=est, random_state=0, n_estimators=50, learning_rate=0.05, loss='square')
ada_model.fit(X_train, y_train)
ada_preds = ada_model.predict(X_test)

In [27]:
# Adaboost quality
print('percentage error', mean_absolute_percentage_error(y_test, ada_preds))
print('absolute error', mean_absolute_error(y_test, ada_preds))
print('root squared error', mean_squared_error(y_test, ada_preds, squared=False))

percentage error 0.08000747315099938
absolute error 81533.19506118582
root squared error 151676.77753796452


In [28]:
# Catboost is fitting automatically during parsing new data in parse_data.py

#from catboost import Pool, CatBoostRegressor
#cat_df = pd.concat([df6,df7,df8], ignore_index=True)
#cat_df.drop(columns=['today']).drop_duplicates(keep='last', inplace=True)
#cat_df = cat_df[cols_x + col_y].dropna()
#cat_df['district'] = le.transform(cat_df['district'])

#X_train, X_test, y_train, y_test = train_test_split(model_df[cols_x], model_df['price'], test_size=0.25, random_state=10)
#train_pool = Pool(X_train, y_train, cat_features=cat_cols)
#test_pool = Pool(X_test, cat_features=cat_cols)
#cb_model = CatBoostRegressor(iterations=3000, learning_rate=0.04, l2_leaf_reg=1, depth=8)
#cb_model.fit(train_pool)
#cb_preds = cb_model.predict(test_pool)

In [29]:
# Load catboost
cb_model = CatBoostRegressor().load_model("cb_model.cbm")

test_pool = Pool(X_test, cat_features=cat_cols)
cb_preds = cb_model.predict(test_pool)

In [30]:
print('percentage error', mean_absolute_percentage_error(y_test, cb_preds))
print('absolute error', mean_absolute_error(y_test, cb_preds))
print('root squared error', mean_squared_error(y_test, cb_preds, squared=False))

percentage error 0.0777342052662733
absolute error 86411.36433352105
root squared error 145560.62535993423


In [31]:
lr_df = pd.DataFrame(data=ada_preds.reshape(-1,1), columns=['ada_preds'])
lr_df['cb_preds'] = cb_preds
lr_df['y'] = y_test.values
train = lr_df[:2500]
test = lr_df[2500:]

In [32]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(train.drop(columns=['y']), train['y'])
lr_preds = lr.predict(test.drop(columns=['y']))

In [33]:
# Check regression quality
print('percentage error', mean_absolute_percentage_error(test['y'], lr_preds))
print('absolute error', mean_absolute_error(test['y'], lr_preds))
print('root squared error', mean_squared_error(test['y'], lr_preds, squared=False))

percentage error 0.07294981681089216
absolute error 83849.53849886722
root squared error 133123.53821969993


In [34]:
# Fit models on full dataset
ada_model.fit(model_df[cols_x], model_df['price'])
ada_preds = ada_model.predict(model_df[cols_x])
cb_preds = cb_model.predict(Pool(model_df[cols_x], cat_features=cat_cols))
new_df = pd.DataFrame(data=ada_preds.reshape(-1,1), columns=['ada_preds'])
new_df['cb_preds'] = cb_preds
new_df['y'] = model_df['price']
lr.fit(new_df.drop(columns=['y']), model_df['price'])

LinearRegression()

In [35]:
# save models to file
import pickle
filename = 'adaboost_regressor.pkl'
pickle.dump(ada_model, open(filename, 'wb'))
#pickle.dump(le, open('district_encode', 'wb'))
filename = 'linear_regressor.pkl'
pickle.dump(lr, open(filename, 'wb'))
#cb_model.save_model('cb_model.cbm')